<a href="https://colab.research.google.com/github/Shivan5h/Minor2/blob/main/Citizen_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install fastapi uvicorn spacy textblob scikit-learn pandas numpy nltk langdetect speechrecognition sentence-transformers python-multipart
!pip -m textblob.download_corpora
!pip -m nltk.downloader punkt
!pip -m spacy download en_core_web_sm



Usage:   
  pip3 <command> [options]

no such option: -m

Usage:   
  pip3 <command> [options]

no such option: -m

Usage:   
  pip3 <command> [options]

no such option: -m


In [3]:
from fastapi import FastAPI, HTTPException, UploadFile, File
from pydantic import BaseModel
import sqlite3
import spacy
from textblob import TextBlob
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from langdetect import detect
import speech_recognition as sr
import hashlib
import os

In [4]:
app = FastAPI()

In [5]:
nlp = spacy.load("en_core_web_sm")
db_path = "citizen_data.db"

In [6]:
# Initialize database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()


In [7]:
cursor.execute('''CREATE TABLE IF NOT EXISTS service (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    address TEXT,
    phone TEXT,
    email TEXT,
    details TEXT
)''')

In [8]:
cursor.execute('''CREATE TABLE IF NOT EXISTS complaint (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    address TEXT,
    phone TEXT,
    email TEXT,
    category TEXT,
    type TEXT,
    details TEXT,
    sentiment TEXT,
    location TEXT,
    summary TEXT,
    response TEXT,
    complaint_hash TEXT
)''')

In [9]:
conn.commit()

In [10]:
class ServiceRequest(BaseModel):
    name: str
    address: str
    phone: str
    email: str
    details: str

In [11]:
class ComplaintRequest(BaseModel):
    name: str
    address: str
    phone: str
    email: str
    details: str

In [12]:
CATEGORIES = {
    "Electricity": ["Street Lightening", "Frequent Cuts", "Frequent Low Voltage"],
    "Water": ["Poor Drainage", "Poor Quality Water", "No availability of water"],
    "Waste": ["Lack of Green Space", "Sewage Issues", "Waste collection service Issues", "Waste accumulated at a place"],
    "Road": ["Potholes", "Maintenance of Road", "Sidewalk Accessibility", "Traffic Congestion", "Public Transport"],
    "Other": ["Public Safety", "School Accessibility", "Healthcare Services", "Public Property Maintenance", "Internet Connectivity", "Air Quality", "Noise Pollution"]
}

In [13]:
def classify_category_type(text):
    for cat, types in CATEGORIES.items():
        for t in types:
            if t.lower() in text.lower():
                return cat, t
    return "Other", "OTH"

In [14]:
def extract_location(text):
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "GPE":
            return ent.text
    return "Unknown"

In [15]:
def analyze_sentiment(text):
    polarity = TextBlob(text).sentiment.polarity
    if polarity > 0.2:
        return "Positive"
    elif polarity < -0.2:
        return "Negative"
    else:
        return "Neutral"

In [16]:
def summarize_text(text):
    blob = TextBlob(text)
    return blob.sentences[0].string if blob.sentences else text

In [18]:
def generate_auto_response(category, type_):
    return f"Your complaint regarding {type_} under {category} category has been registered and will be addressed shortly."


In [17]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return "Unknown"

In [19]:
def complaint_exists(details):
    hash_val = hashlib.sha256(details.encode()).hexdigest()
    cursor.execute("SELECT 1 FROM complaint WHERE complaint_hash=?", (hash_val,))
    return cursor.fetchone() is not None, hash_val

In [20]:
def get_similar_complaints(text):
    df = pd.read_sql_query("SELECT id, details FROM complaint", conn)
    if df.empty:
        return []
    vect = TfidfVectorizer()
    vectors = vect.fit_transform(df['details'].tolist() + [text])
    sims = cosine_similarity(vectors[-1:], vectors[:-1])[0]
    df['similarity'] = sims
    similar_df = df[df['similarity'] > 0.5]
    return similar_df['id'].tolist()

In [21]:
@app.post("/request_service/")
def request_service(request: ServiceRequest):
    cursor.execute("INSERT INTO service (name, address, phone, email, details) VALUES (?, ?, ?, ?, ?)",
                   (request.name, request.address, request.phone, request.email, request.details))
    conn.commit()
    return {"message": "Service request submitted successfully."}


In [22]:
@app.post("/raise_complaint/")
def raise_complaint(request: ComplaintRequest):
    is_duplicate, hash_val = complaint_exists(request.details)
    if is_duplicate:
        raise HTTPException(status_code=400, detail="Duplicate complaint detected.")

    category, type_ = classify_category_type(request.details)
    sentiment = analyze_sentiment(request.details)
    location = extract_location(request.details)
    summary = summarize_text(request.details)
    response = generate_auto_response(category, type_)

    cursor.execute("""
        INSERT INTO complaint (name, address, phone, email, category, type, details, sentiment, location, summary, response, complaint_hash)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)""",
        (request.name, request.address, request.phone, request.email, category, type_, request.details, sentiment, location, summary, response, hash_val))
    conn.commit()

    similar = get_similar_complaints(request.details)

    return {
        "message": "Complaint registered successfully.",
        "category": category,
        "type": type_,
        "sentiment": sentiment,
        "location": location,
        "summary": summary,
        "auto_response": response,
        "similar_complaints": similar
    }


In [27]:
@app.post("/speech_to_text/")
def speech_to_text(audio: UploadFile = File(...)):
    recognizer = sr.Recognizer()
    audio_path = f"temp_audio_{audio.filename}"
    with open(audio_path, "wb") as f:
        f.write(audio.file.read())

    with sr.AudioFile(audio_path) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data)
        except sr.UnknownValueError:
            text = "Could not understand the audio."
    os.remove(audio_path)
    return {"transcription": text}
